<a href="https://colab.research.google.com/github/hvarS/CS60075-Team28-Task-1/blob/main/corpus_features_extraction_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy
!pip install python-datamuse
!pip install stanza

import stanza

corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir
!ls $CORENLP_HOME


!pip install nltk

  Created wheel for python-datamuse: filename=python_datamuse-1.3.0-cp37-none-any.whl size=4477 sha256=2e1a22363cff08284fc4cbf42090cac41373f42ac5d17c709d50c5c147249e6c
  Stored in directory: /root/.cache/pip/wheels/29/03/80/d8f9fd80e72813c3559f6a37ffa6d9fdda6116d56a99011ded
Successfully built python-datamuse
     |████████████████████████████████| 286kB 14.1MB/s 


2021-04-08 19:48:52 INFO: Installing CoreNLP package into ./corenlp...
2021-04-08 19:50:33 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=./corenlp`.


build.xml				  jollyday.jar
corenlp.sh				  LIBRARY-LICENSES
CoreNLP-to-HTML.xsl			  LICENSE.txt
ejml-core-0.39.jar			  Makefile
ejml-core-0.39-sources.jar		  patterns
ejml-ddense-0.39.jar			  pom-java-11.xml
ejml-ddense-0.39-sources.jar		  pom.xml
ejml-simple-0.39.jar			  protobuf.jar
ejml-simple-0.39-sources.jar		  README.txt
input.txt				  RESOURCE-LICENSES
input.txt.out				  SemgrexDemo.java
input.txt.xml				  ShiftReduceDemo.java
javax.activation-api-1.2.0.jar		  slf4j-api.jar
javax.activation-api-1.2.0-sources.jar	  slf4j-simple.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.2.0.jar
javax.json.jar				  stanford-corenlp-4.2.0-javadoc.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.2.0-models.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   stanford-corenlp-4.2.0-sources.jar
jaxb-core-2.3.0.1.jar			  StanfordCoreNlpDemo.java
jaxb-core-2.3.0.1-sources.jar		  StanfordDependenciesManual.pdf
jaxb-impl-2.4.0-b180830.0438.jar	  sutime
jaxb-impl-2.4.0-b180830.0438-sources

In [ ]:
# basepath = '/content/drive/MyDrive/CS60075-Team28-Task-1/data/preprocessed/lcp_single_train_preprocessed.csv'
# basepath = '/content/drive/MyDrive/CS60075-Team28-Task-1/data/preprocessed/lcp_single_test_preprocessed.csv'

basepaths = ['/content/CS60075-Team28-Task-1/data/preprocessed/lcp_multi_train_preprocessed.csv',
'/content/drive/MyDrive/CS60075-Team28-Task-1/data/preprocessed/lcp_multi_test_preprocessed.csv']


df = pd.read_csv(basepaths[1])
df.head()

,id,corpus,sentence,token
0,3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,bible,come intending bring bound chief priest,chief priest
1,302U8RURJZ1WF35NXY44RD66WL4NVH,bible,day lord take away beauty anklet headband cres...,crescent necklace
2,3UDTAB6HH6ZVX00DTRXAOJLWX0B094,bible,unclean shall take ash burning sin offering ru...,sin offering
3,3L2OEKSTW9ASGQDOW725GFK5P77Y8D,bible,precious treasure oil dwelling wise foolish ma...,precious treasure
4,39N6W9XWRDN795J6F5ET8S13DQKYGT,bible,long god shall adversary reproach,adversary reproach


In [ ]:
df.dtypes

id          object
corpus      object
sentence    object
token       object
dtype: object

In [ ]:
df[['token1','token2']] = df['token'].str.split(' ',expand=True)
df['token_length'] = df['token'].str.len()
df.head()

,id,corpus,sentence,token,token1,token2,token_length
0,3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,bible,come intending bring bound chief priest,chief priest,chief,priest,12
1,302U8RURJZ1WF35NXY44RD66WL4NVH,bible,day lord take away beauty anklet headband cres...,crescent necklace,crescent,necklace,17
2,3UDTAB6HH6ZVX00DTRXAOJLWX0B094,bible,unclean shall take ash burning sin offering ru...,sin offering,sin,offering,12
3,3L2OEKSTW9ASGQDOW725GFK5P77Y8D,bible,precious treasure oil dwelling wise foolish ma...,precious treasure,precious,treasure,17
4,39N6W9XWRDN795J6F5ET8S13DQKYGT,bible,long god shall adversary reproach,adversary reproach,adversary,reproach,18


In [ ]:
#function to obtain syablles for words
from datamuse import datamuse
api = datamuse.Datamuse()

def get_syllables(word):
    syllables = 0
    word_results = api.words(sp=word, max=1, md='psf')
    if len(word_results)>0: 
        word = word_results[0]["word"]
        syllables = int(word_results[0]["numSyllables"])
    return syllables

df['syllables'] = df['token1'].apply(lambda x: get_syllables(x) ) + df['token2'].apply(lambda x: get_syllables(x) ) 
df.head()

,id,corpus,sentence,token,token1,token2,token_length,syllables
0,3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,bible,come intending bring bound chief priest,chief priest,chief,priest,12,2
1,302U8RURJZ1WF35NXY44RD66WL4NVH,bible,day lord take away beauty anklet headband cres...,crescent necklace,crescent,necklace,17,4
2,3UDTAB6HH6ZVX00DTRXAOJLWX0B094,bible,unclean shall take ash burning sin offering ru...,sin offering,sin,offering,12,4
3,3L2OEKSTW9ASGQDOW725GFK5P77Y8D,bible,precious treasure oil dwelling wise foolish ma...,precious treasure,precious,treasure,17,4
4,39N6W9XWRDN795J6F5ET8S13DQKYGT,bible,long god shall adversary reproach,adversary reproach,adversary,reproach,18,6


In [ ]:
temp = df[['sentence','id']].copy()
temp.head()

,sentence,id
0,come intending bring bound chief priest,3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL
1,day lord take away beauty anklet headband cres...,302U8RURJZ1WF35NXY44RD66WL4NVH
2,unclean shall take ash burning sin offering ru...,3UDTAB6HH6ZVX00DTRXAOJLWX0B094
3,precious treasure oil dwelling wise foolish ma...,3L2OEKSTW9ASGQDOW725GFK5P77Y8D
4,long god shall adversary reproach,39N6W9XWRDN795J6F5ET8S13DQKYGT


In [ ]:
from stanza.server import CoreNLPClient

############## uncomment to start the client

# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed




client = CoreNLPClient(
    annotators=['pos', 'depparse'], 
    memory='4G', 
    endpoint='http://localhost:9005',
    output_format="json",
    be_quiet=True)

print(client)
client.start()


import time; time.sleep(10)

def parse(string):
    output = client.annotate(string)
    return output

temp['parse'] = temp['sentence'].apply(lambda x: parse(x))
temp['parse'].head()

2021-04-08 19:51:41 INFO: Writing properties to tmp file: corenlp_server-b9a869058bee4c1f.props
2021-04-08 19:51:41 INFO: Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9005 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-b9a869058bee4c1f.props -annotators pos,depparse -preload -outputFormat json


0    {'sentences': [{'index': 0, 'basicDependencies...
1    {'sentences': [{'index': 0, 'basicDependencies...
2    {'sentences': [{'index': 0, 'basicDependencies...
3    {'sentences': [{'index': 0, 'basicDependencies...
4    {'sentences': [{'index': 0, 'basicDependencies...
Name: parse, dtype: object

In [ ]:
# get pos of the token (here phrase) from the parse constructed by the corenlp client. same for the dependencies
import string

remove = string.punctuation
remove = remove.replace("-", "")
remove = remove.replace("'", "")# don't remove apostraphies 
remove = remove + '“'
remove = remove +'”'
def get_pos(row,token):
    # should be token
    word = row[token]

    parse = row['parse']

    # below loop is okay i guess
    for i in range(len(parse['sentences'][0]['tokens'])):

        comp_word = parse['sentences'][0]['tokens'][i]['word']
        comp_word = comp_word.lower()
        comp_word = comp_word.translate({ord(char): None for char in remove})

        if comp_word == word:
            return parse['sentences'][0]['tokens'][i]['pos']

def get_dep(row,token):
    number = 0
    word = row[token]
    parse = row['parse']
    for i in range(len(parse['sentences'][0]['basicDependencies'])):
        comp_word = parse['sentences'][0]['basicDependencies'][i]['governorGloss']
        comp_word = comp_word.lower()
        comp_word = comp_word.translate({ord(char): None for char in remove})

        if comp_word==word:
            number += 1

    return number

word_parse_features = pd.merge(temp, df)


word_parse_features['pos1'] = word_parse_features.apply(get_pos,token = 'token1', axis=1)
word_parse_features['dep num1'] = word_parse_features.apply(get_dep,token = 'token1', axis=1)


word_parse_features['pos2'] = word_parse_features.apply(get_pos,token = 'token2', axis=1)
word_parse_features['dep num2'] = word_parse_features.apply(get_dep,token = 'token2', axis=1)



word_parse_features.head()

,sentence,id,parse,corpus,token,token1,token2,token_length,syllables,pos1,dep num1,pos2,dep num2
0,come intending bring bound chief priest,3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,"{'sentences': [{'index': 0, 'basicDependencies...",bible,chief priest,chief,priest,12,2,JJ,0,NN,2
1,day lord take away beauty anklet headband cres...,302U8RURJZ1WF35NXY44RD66WL4NVH,"{'sentences': [{'index': 0, 'basicDependencies...",bible,crescent necklace,crescent,necklace,17,4,NN,1,NN,2
2,unclean shall take ash burning sin offering ru...,3UDTAB6HH6ZVX00DTRXAOJLWX0B094,"{'sentences': [{'index': 0, 'basicDependencies...",bible,sin offering,sin,offering,12,4,NN,3,VBG,1
3,precious treasure oil dwelling wise foolish ma...,3L2OEKSTW9ASGQDOW725GFK5P77Y8D,"{'sentences': [{'index': 0, 'basicDependencies...",bible,precious treasure,precious,treasure,17,4,JJ,0,NN,1
4,long god shall adversary reproach,39N6W9XWRDN795J6F5ET8S13DQKYGT,"{'sentences': [{'index': 0, 'basicDependencies...",bible,adversary reproach,adversary,reproach,18,6,NN,0,NN,4


In [ ]:
word_parse_features.to_csv('/content/drive/MyDrive/cwi2018_traindevset/1.csv')

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
print(wordnet.synsets('room'))

[Synset('room.n.01'), Synset('room.n.02'), Synset('room.n.03'), Synset('room.n.04'), Synset('board.v.02')]


In [ ]:
def get_wordnet_pos(treebank_tag):
    # print(treebank_tag)
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatiser(row,token):

    word = row['token' + token]
    pos = row['pos' + token]
    
    # print(word,pos)
    try:
        lemma = wordnet_lemmatizer.lemmatize(word, pos = get_wordnet_pos(pos))
        # lemma = wordnet_lemmatizer.lemmatize(word)
        return lemma
    except:
        try:
            lemma = wordnet_lemmatizer.lemmatize(word)
            return lemma
        except:
            print(word)

word_parse_features['lemma1'] = word_parse_features.apply(lemmatiser,token = '1', axis=1)
word_parse_features['lemma2'] = word_parse_features.apply(lemmatiser,token = '2', axis=1)
word_parse_features.head()

,sentence,id,parse,corpus,token,token1,token2,token_length,syllables,pos1,dep num1,pos2,dep num2,lemma1,lemma2
0,come intending bring bound chief priest,3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,"{'sentences': [{'index': 0, 'basicDependencies...",bible,chief priest,chief,priest,12,2,JJ,0,NN,2,chief,priest
1,day lord take away beauty anklet headband cres...,302U8RURJZ1WF35NXY44RD66WL4NVH,"{'sentences': [{'index': 0, 'basicDependencies...",bible,crescent necklace,crescent,necklace,17,4,NN,1,NN,2,crescent,necklace
2,unclean shall take ash burning sin offering ru...,3UDTAB6HH6ZVX00DTRXAOJLWX0B094,"{'sentences': [{'index': 0, 'basicDependencies...",bible,sin offering,sin,offering,12,4,NN,3,VBG,1,sin,offer
3,precious treasure oil dwelling wise foolish ma...,3L2OEKSTW9ASGQDOW725GFK5P77Y8D,"{'sentences': [{'index': 0, 'basicDependencies...",bible,precious treasure,precious,treasure,17,4,JJ,0,NN,1,precious,treasure
4,long god shall adversary reproach,39N6W9XWRDN795J6F5ET8S13DQKYGT,"{'sentences': [{'index': 0, 'basicDependencies...",bible,adversary reproach,adversary,reproach,18,6,NN,0,NN,4,adversary,reproach


In [ ]:
def synonyms(word):
    synonyms=0
    try:
        results = wordnet.synsets(word)
        synonyms = len(results)
        return synonyms
    except:
        return synonyms

def hypernyms(word):
    hypernyms=0
    try:
        results = wordnet.synsets(word)
        hypernyms = len(results[0].hypernyms())
        return hypernyms
    except:
        return hypernyms

def hyponyms(word):
    hyponyms=0
    try:
        results = wordnet.synsets(word)
    except:
        return hyponyms
    try:
        hyponyms = len(results[0].hyponyms())
        return hyponyms
    except:
        return hyponyms

In [ ]:
word_parse_features['synonyms1'] = word_parse_features['lemma1'].apply(lambda x: synonyms(x))
word_parse_features['hypernyms1'] = word_parse_features['lemma1'].apply(lambda x: hypernyms(x))
word_parse_features['hyponyms1'] = word_parse_features['lemma1'].apply(lambda x: hyponyms(x))

word_parse_features['synonyms2'] = word_parse_features['lemma2'].apply(lambda x: synonyms(x))
word_parse_features['hypernyms2'] = word_parse_features['lemma2'].apply(lambda x: hypernyms(x))
word_parse_features['hyponyms2'] = word_parse_features['lemma2'].apply(lambda x: hyponyms(x))

word_parse_features.head()

,sentence,id,parse,corpus,token,token1,token2,token_length,syllables,pos1,dep num1,pos2,dep num2,lemma1,lemma2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2
0,come intending bring bound chief priest,3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,"{'sentences': [{'index': 0, 'basicDependencies...",bible,chief priest,chief,priest,12,2,JJ,0,NN,2,chief,priest,4,1,10,2,2,11
1,day lord take away beauty anklet headband cres...,302U8RURJZ1WF35NXY44RD66WL4NVH,"{'sentences': [{'index': 0, 'basicDependencies...",bible,crescent necklace,crescent,necklace,17,4,NN,1,NN,2,crescent,necklace,2,1,0,1,1,3
2,unclean shall take ash burning sin offering ru...,3UDTAB6HH6ZVX00DTRXAOJLWX0B094,"{'sentences': [{'index': 0, 'basicDependencies...",bible,sin offering,sin,offering,12,4,NN,3,VBG,1,sin,offer,8,1,1,16,1,0
3,precious treasure oil dwelling wise foolish ma...,3L2OEKSTW9ASGQDOW725GFK5P77Y8D,"{'sentences': [{'index': 0, 'basicDependencies...",bible,precious treasure,precious,treasure,17,4,JJ,0,NN,1,precious,treasure,5,0,0,6,1,4
4,long god shall adversary reproach,39N6W9XWRDN795J6F5ET8S13DQKYGT,"{'sentences': [{'index': 0, 'basicDependencies...",bible,adversary reproach,adversary,reproach,18,6,NN,0,NN,4,adversary,reproach,1,1,4,3,1,2


In [ ]:
# word_parse_features.to_csv('/content/drive/MyDrive/cwi2018_traindevset/2.csv')

In [ ]:

def is_noun(tag):
    return tag in ['NN', 'NNS', 'NNP', 'NNPS']


def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']


def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']



def penn_to_google(tag):
    if is_adjective(tag):
        return 'adj'
    elif is_noun(tag):
        return 'n'
    elif is_adverb(tag):
        return 'adv'
    elif is_verb(tag):
        return 'v'
    return None

def get_frequency(row,token):
    nofreq = float(0.000000)
    word = row["token"+token]
    word = str(word)
    tag = row["pos"+token]
    tag = penn_to_google(tag)

    try:
        word_results = api.words(sp=word, max=1, md='pf')
        tag_list = (word_results[0]['tags'][:-1])

        frequency = word_results[0]['tags'][-1][2:]

        frequency = float(frequency)

        if tag in tag_list :
            return frequency
        else:
            lemma = row['lemma'+token]
            try:
                word_results = api.words(sp=lemma, max=1, md='pf')
                tag_list = (word_results[0]['tags'][:-1])

                frequency = word_results[0]['tags'][-1][2:]

                frequency = float(frequency)

                if tag in tag_list:
                    return frequency
                else:
                    return nofreq
            except:
                return nofreq

    except:


        return nofreq 


In [ ]:
word_parse_features['google frequency1'] = word_parse_features.apply(get_frequency,token = '1' ,axis=1)
word_parse_features['google frequency2'] = word_parse_features.apply(get_frequency,token = '2' ,axis=1)

In [ ]:
word_parse_features.head()

,sentence,id,parse,corpus,token,token1,token2,token_length,syllables,pos1,dep num1,pos2,dep num2,lemma1,lemma2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2,google frequency1,google frequency2
0,come intending bring bound chief priest,3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,"{'sentences': [{'index': 0, 'basicDependencies...",bible,chief priest,chief,priest,12,2,JJ,0,NN,2,chief,priest,4,1,10,2,2,11,0.000000,39.121551
1,day lord take away beauty anklet headband cres...,302U8RURJZ1WF35NXY44RD66WL4NVH,"{'sentences': [{'index': 0, 'basicDependencies...",bible,crescent necklace,crescent,necklace,17,4,NN,1,NN,2,crescent,necklace,2,1,0,1,1,3,4.830131,4.021996
2,unclean shall take ash burning sin offering ru...,3UDTAB6HH6ZVX00DTRXAOJLWX0B094,"{'sentences': [{'index': 0, 'basicDependencies...",bible,sin offering,sin,offering,12,4,NN,3,VBG,1,sin,offer,8,1,1,16,1,0,51.783901,114.840014
3,precious treasure oil dwelling wise foolish ma...,3L2OEKSTW9ASGQDOW725GFK5P77Y8D,"{'sentences': [{'index': 0, 'basicDependencies...",bible,precious treasure,precious,treasure,17,4,JJ,0,NN,1,precious,treasure,5,0,0,6,1,4,23.227710,14.658214
4,long god shall adversary reproach,39N6W9XWRDN795J6F5ET8S13DQKYGT,"{'sentences': [{'index': 0, 'basicDependencies...",bible,adversary reproach,adversary,reproach,18,6,NN,0,NN,4,adversary,reproach,1,1,4,3,1,2,5.556080,4.950713


In [ ]:
word_parse_features.to_csv('/content/drive/MyDrive/CS60075-Team28-Task-1/data/extracted_features/lcp_multi_test_split_features.csv')

In [ ]:
paths = ['/content/drive/MyDrive/CS60075-Team28-Task-1/data/extracted_features/lcp_multi_test_split_features.csv','/content/drive/MyDrive/CS60075-Team28-Task-1/data/extracted_features/lcp_multi_train_split_features.csv']





for path in paths:
    df2 = pd.read_csv(path)

    def get_vowels(word):
        val = 0

        for w in word:
            if(w in ['A', 'a', 'E', 'e', 'I', 'i', 'O', 'o', 'U','u']):
                val+=1
        return val

    df2['token_vowels'] = df2['token'].apply(lambda x: get_vowels(x) )
    df2.to_csv(path)
    df2.head(10)